In [1]:
import pandas as pd
import pyspark.pandas as ps
import requests
import json

from sqlalchemy import create_engine
from pca import pca
from pyspark.sql.functions import pandas_udf, PandasUDFType, max, col, countDistinct, when, rank, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.window import Window

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import ClusterCentroids

# from backend_spark.doris_common.doris_client import DorisClient
from functools import reduce
from pyspark.sql import DataFrame
from typing import Optional

In [2]:
import os 
from pyspark.sql import SparkSession

os.environ['PYSPARK_PYTHON'] = '/usr/local/python-3.9.13/bin/python3'

spark = SparkSession.builder \
    .appName("pandas_udf") \
    .config('spark.sql.session.timeZone', 'Asia/Shanghai') \
    .config("spark.scheduler.mode", "FAIR") \
    .config('spark.driver.memory', '1024m') \
    .config('spark.driver.cores', '3') \
    .config('spark.executor.memory', '1024m') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '2') \
    .config('spark.driver.host','192.168.22.28') \
    .master("spark://192.168.12.47:7077,192.168.12.48:7077") \
    .getOrCreate()

In [3]:
###################################################################
##########################融合OPER_NO字段##########################
###################################################################
def integrate_operno(df, merge_operno_list):
    if merge_operno_list is not None and len(merge_operno_list) > 0:
        # 将mergeOperno中每个字典的values提取出来，组成一个列表
        values_to_replace = [list(rule.values())[0] for rule in merge_operno_list]

        # 将每一个字典中的values拼接起来
        merged_values = ["_".join(list(rule.values())[0]) for rule in merge_operno_list]

        for values, replacement_value in zip(values_to_replace, merged_values):
            df = df.withColumn("OPER_NO",
                               when(col("OPER_NO").isin(values), replacement_value).otherwise(col("OPER_NO")))
        return df
    else:
        return df


############################################################################
##################################FDC数据预处理###############################
############################################################################
def _pre_process(df):
    """
    param df: 从数据库中读取出来的某个CASE数据
    return: 数据预处理，后面要根据实际情况统一添加
    """
    # 只选出会用到的列
    df = df.select('WAFER_ID', 'TOOL_ID', 'RUN_ID', 'EQP_NAME', 'PRODUCT_ID', 'PRODG1', 'TOOL_NAME',
                   'OPER_NO', 'parametric_name', 'STATISTIC_RESULT', 'label')
    # 剔除NA值
    df = df.filter(col('STATISTIC_RESULT').isNotNull())
    # 按照所有的行进行去重
    df1 = df.dropDuplicates()
    # 选最新的RUN
    df2 = df1.groupBy('WAFER_ID', 'OPER_NO', 'TOOL_ID').agg(max('RUN_ID').alias('RUN_ID'))
    df_run = df1.join(df2.dropDuplicates(subset=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID']),
                      on=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID'], how='inner')
    return df_run


def commonality_analysis(df_run, grpby_list):
    """
    param df_run: 数据预处理后的数据
    return: 共性分析后的结果， 返回bad wafer前十的组合
    """
    grps = (df_run.groupBy(grpby_list)
            .agg(countDistinct('WAFER_ID').alias('wafer_count'),
                 countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
                 countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
            .orderBy('bad_num', ascending=False))

    # 单站点+单腔室的情况
    if grps.count() == 1:
        return grps
    else:
        grps = grps.filter(grps['bad_num'] > 0)
        window_sep = Window().orderBy(col("bad_num").desc())
        ranked_df = grps.withColumn("rank", rank().over(window_sep))
        grpss = ranked_df.filter(col("rank") <= 10).drop("rank")
        return grpss


###########################################################################
#################################获取样本数据#########################
############################################################################
def get_data_list(common_res, grpby_list, big_or_small='big'):
    """
    param common_res: 共性分析后的结果, 按照大样本或者小样本条件筛选出组合
    param grpby_list: 按照PRODG1+OPER_NO+TOOL_NAME分组或OPER_NO+TOOL_NAME分组
    param big_or_small: big或者small
    return: 对应组合的字典形式, 包在一个大列表中
    """
    assert big_or_small in ['big', 'small'], "只能选择big或者small, 请检查拼写"
    if big_or_small == 'big':
        good_bad_grps = common_res.filter("good_num >= 3 AND bad_num >= 3")
    else:
        good_bad_grps = common_res.filter("bad_num >= 1 AND wafer_count >=2")
    good_bad_grps = good_bad_grps.orderBy(col("bad_num").desc(), col("wafer_count").desc(),
                                          col("good_num").desc()).limit(5)

    if 'PRODG1' in grpby_list:
        data_list = good_bad_grps['PRODG1', 'OPER_NO', 'EQP_NAME'].collect()
    else:
        data_list = good_bad_grps['OPER_NO', 'EQP_NAME'].collect()

    data_dict_list = [row.asDict() for row in data_list]
    return data_dict_list


def get_train_data(df_run, data_dict_list):
    """
    param df_run: 数据预处理后的数据
    param data_dict: 筛选后的字典结果
    return: 从原始数据中过滤出真正用来建模的组合数据
    """
    if len(data_dict_list[0]) == 3:
        prod, oper, tool = data_dict_list[0]['PRODG1'], data_dict_list[0]['OPER_NO'], data_dict_list[0]['EQP_NAME']
        df_s = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' AND EQP_NAME == '{}'".format(prod, oper, tool))
        for i in range(1, len(data_dict_list)):
            prod, oper, tool = data_dict_list[i]['PRODG1'], data_dict_list[i]['OPER_NO'], data_dict_list[i]['EQP_NAME']
            df_m = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' and EQP_NAME == '{}'".format(prod, oper, tool))
            df_s = df_s.union(df_m)
    else:
        oper, tool = data_dict_list[0]['OPER_NO'], data_dict_list[0]['EQP_NAME']
        df_s = df_run.filter("OPER_NO == '{}' AND EQP_NAME == '{}'".format(oper, tool))
        for i in range(1, len(data_dict_list)):
            oper, tool = data_dict_list[i]['OPER_NO'], data_dict_list[i]['EQP_NAME']
            df_m = df_run.filter("OPER_NO == '{}' and EQP_NAME == '{}'".format(oper, tool))
            df_s = df_s.union(df_m)
    return df_s


############################################################################
#########################获取传入的整个数据中的所有bad_wafer个数############
############################################################################
def get_all_bad_wafer_num(df):
    """
    param df: 筛选后的数据
    return: 数据中所有bad_wafer的数量
    """
    return df.filter("label == 1").select('WAFER_ID').distinct().count()


############################################################################
#####################对good>=3和bad>=3的数据，用rf建模######################
############################################################################
def get_pivot_table(df, by):
    """
    param df: 大样本组合的数据
    param by: 分组字段
    return: 表格透视后的结果
    """
    if len(by) == 3:
        df_pivot = df.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'],
                                                 columns=['OPER_NO', 'EQP_NAME', 'parametric_name', 'PRODG1'],
                                                 values=['STATISTIC_RESULT'])
    else:
        df_pivot = df.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'],
                                                 columns=['OPER_NO', 'EQP_NAME', 'parametric_name'],
                                                 values=['STATISTIC_RESULT'])
    df_pivot.columns = df_pivot.columns.map('#'.join)
    df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)
    return df_pivot


def fit_rf_big_sample(df, by):
    """
    param df: 大样本组合的数据
    param by: 分组字段
    return: RandomForest建模后的结果
    """
    schema_all = StructType([
        StructField("PRODG1", StringType(), True),
        StructField("OPER_NO", StringType(), True),
        StructField("EQP_NAME", StringType(), True),
        StructField("bad_wafer", IntegerType(), True),
        StructField("roc_auc_score", FloatType(), True),
        StructField("features", StringType(), True),
        StructField("importance", FloatType(), True)])

    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_model_result(df_run):
        # 表格透视
        df_pivot = get_pivot_table(df=df_run, by=by)

        # 定义自变量和因变量
        X_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]
        y_train = df_pivot[['label']]

        z_ratio = y_train.value_counts(normalize=True)
        good_ratio = z_ratio[0]
        bad_ratio = z_ratio[1]
        if abs(good_ratio - bad_ratio) > 0.7:
            undersampler = ClusterCentroids(random_state=101)
            X_train, y_train = undersampler.fit_resample(X_train, y_train)

        # 网格搜索
        pipe = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
            ('scaler', StandardScaler()),
            ('model', RandomForestClassifier())])
        param_grid = {'model__n_estimators': [*range(50, 100, 10)],
                      'model__max_depth': [*range(10, 50, 10)]}
        grid = GridSearchCV(estimator=pipe, scoring='roc_auc', param_grid=param_grid, cv=3, n_jobs=-1)
        grid.fit(X_train.values, y_train.values.ravel())
        roc_auc_score_ = grid.best_score_

        # 特征重要度、结果汇总
        small_importance_res = pd.DataFrame({
            'features': X_train.columns,
            'importance': grid.best_estimator_.steps[2][1].feature_importances_}).sort_values(by='importance',
                                                                                              ascending=False)
        if len(by) == 3:
            small_sample_res = pd.DataFrame({
                'PRODG1': df_run['PRODG1'].unique(),
                'OPER_NO': df_run['OPER_NO'].unique(),
                'EQP_NAME': df_run['EQP_NAME'].unique(),
                'bad_wafer': sum(df_pivot['label']),
                'roc_auc_score': roc_auc_score_})
        else:
            PRODG1 = 'grplen2'
            small_sample_res = pd.DataFrame({
                'PRODG1': PRODG1,
                'OPER_NO': df_run['OPER_NO'].unique(),
                'EQP_NAME': df_run['EQP_NAME'].unique(),
                'bad_wafer': sum(df_pivot['label']),
                'roc_auc_score': roc_auc_score_})
        return pd.concat([small_importance_res, small_sample_res])

    return df.groupby(by).apply(get_model_result)


#####################################################################################
#########################对good>=3和bad>=3建模后的结果进行整合############################
#####################################################################################
def split_score_big_sample(df, by):
    """
    param df: RandomForest建模后的结果
    param by: 分组字段
    return: roc_auc分数结果
    """
    schema_all = StructType([StructField("PRODG1", StringType(), True),
                             StructField("OPER_NO", StringType(), True),
                             StructField("EQP_NAME", StringType(), True),
                             StructField("bad_wafer", IntegerType(), True),
                             StructField("roc_auc_score", FloatType(), True)])

    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_result(model_results):
        sample_res = model_results[['PRODG1', 'OPER_NO', 'EQP_NAME', 'bad_wafer', 'roc_auc_score']].dropna(axis=0)
        sample_res = sample_res[sample_res['roc_auc_score'] > 0.6]
        return sample_res

    return df.groupby(by).apply(get_result)


def split_features(df, index) -> str:
    """
    param df: RandomForest建模后的feature_importance_table
    param index: 顺序值
    return: 字段属性值
    """
    return df['features'].apply(lambda x: x.split('#')[index])


def get_split_feature_importance_table(df, by):
    """
    param df: RandomForest建模后的feature_importance_table
    param by: OPER_NO+TOOL_NAME+PRODG1或者OPER_NO+TOOL_NAME
    return: 分裂features后的表
    """
    df['STATISTIC_RESULT'] = split_features(df, 0)
    df['OPER_NO'] = split_features(df, 1)
    df['EQP_NAME'] = split_features(df, 2)
    df['parametric_name'] = split_features(df, 3)
    df['step'] = split_features(df, 4)
    df['stats'] = split_features(df, 5)

    if 'PRODG1' in by:
        df['PRODG1'] = split_features(df, 6)
    else:
        df = df.assign(PRODG1='grplen2')

    df = df.drop(['features', 'STATISTIC_RESULT'], axis=1).reset_index(drop=True)
    return df


def add_feature_stats(df):
    """
    param df: 经过处理后的feature_importance_table
    return: 新增一列，含有参数的所有统计特征:feature_stats
    """
    feature_stats = df.groupby(['PRODG1', 'OPER_NO', 'EQP_NAME', 'parametric_name', 'step'])[
        'stats'].unique().reset_index()
    feature_stats['stats'] = [feature_stats['stats'].iloc[i].tolist() for i in range(len(feature_stats))]
    feature_stats['stats'] = feature_stats['stats'].apply(lambda x: "#".join(x))
    feature_stats = feature_stats.assign(parametric_name=lambda x: x['parametric_name'] + str('#') + x['step']).drop(
        'step', axis=1)
    return feature_stats


def split_calculate_features_big_sample(df, by):
    """
    param df: RandomForest建模后的结果
    param by: 分组字段
    return: features和importance结果
    """
    schema_all = StructType([
        StructField("PRODG1", StringType(), True),
        StructField("OPER_NO", StringType(), True),
        StructField("EQP_NAME", StringType(), True),
        StructField("parametric_name", StringType(), True),
        StructField("importance", FloatType(), True),
        StructField("stats", StringType(), True)])

    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_result(model_results):
        # 先从随机森林的模型结果中取出包含features和importance的dataframe
        feature_importance_table = model_results[['features', 'importance']].dropna(axis=0)

        # 分裂features
        feature_importance_res_split = get_split_feature_importance_table(feature_importance_table, by)

        # 去除importance为0的组合
        feature_importance_res_split_drop = feature_importance_res_split.query("importance > 0").reset_index(drop=True)

        # 取每一种组合结果的前60%或者100%
        feature_importance_res_split_nlargest = (
            feature_importance_res_split_drop.groupby(by=['PRODG1', 'OPER_NO', 'EQP_NAME'])
            .apply(lambda x: x.nlargest(int(x.shape[0] * 0.6), 'importance') if x.shape[0] > 1 else x.nlargest(
                int(x.shape[0] * 1), 'importance'))
            .reset_index(drop=True))

        # 新增一列，含有参数的所有统计特征:feature_stats
        feature_stats = add_feature_stats(feature_importance_res_split_drop)

        # 对同一种组合里的同一个参数进行求和:feature_importance_groupby
        feature_importance_groupby = (feature_importance_res_split_nlargest.groupby(['PRODG1', 'OPER_NO', 'EQP_NAME',
                                                                                     'parametric_name', 'step'])[
                                          'importance'].sum().reset_index())
        feature_importance_groupby = feature_importance_groupby.assign(
            parametric_name=lambda x: x['parametric_name'] + str('#') + x['step']).drop('step', axis=1)

        # feature_stats和feature_importance_groupby连接
        grpby_stats = pd.merge(feature_stats, feature_importance_groupby,
                               on=['PRODG1', 'OPER_NO', 'EQP_NAME', 'parametric_name']).dropna().reset_index(drop=True)
        return grpby_stats

    return df.groupby(by).apply(get_result)


def get_finall_results_big_sample(s_res, f_res, bad_wafer_num):
    """
    param s_res: roc_auc分数结果
    param f_res: features和importance结果
    param bad_wafer_num: 数据中所有bad_wafer的数量
    return: 最后的建模结果
    """
    # feature_importance_groupby和sample_res连接
    roc_auc_score_all = s_res.agg({"roc_auc_score": "sum"}).collect()[0][0]
    s_res = s_res.withColumn("roc_auc_score_ratio", col("roc_auc_score") / roc_auc_score_all)
    s_res = s_res.withColumn("bad_ratio", col("bad_wafer") / bad_wafer_num)

    df_merge = s_res.join(f_res, on=['PRODG1', 'OPER_NO', 'EQP_NAME'], how='left')
    df_merge = df_merge.withColumn('weight_original', col('roc_auc_score_ratio') * col('bad_ratio') * col('importance'))

    # 最后再次进行一次归一化
    weight_all = df_merge.agg({"weight_original": "sum"}).collect()[0][0]
    df_merge = df_merge.withColumn("weight", col("weight_original") / weight_all)

    df_merge = df_merge.select(['PRODG1', 'OPER_NO', 'EQP_NAME',
                                'parametric_name', 'weight', 'stats']).orderBy('weight', ascending=False)
    return df_merge


#####################################################################################
#############################将建模后的结果增加特定的列####################################
#####################################################################################
def add_certain_column(df, by, request_id):
    """
    param df: 最后的建模结果
    param by: 分组字段, 手动增加一列add
    param request_id: 传入的request_id
    return: 最后的建模结果增加特定的列
    """
    schema_all = StructType([
        StructField("PRODG1", StringType(), True),
        StructField("OPER_NO", StringType(), True),
        StructField("EQP_NAME", StringType(), True),
        StructField("stats", StringType(), True),
        StructField("parametric_name", StringType(), True),
        StructField("weight", FloatType(), True),
        StructField("request_id", StringType(), True),
        StructField("weight_percent", FloatType(), True),
        StructField("index_no", IntegerType(), True)])

    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_result(final_res):
        final_res['weight'] = final_res['weight'].astype(float)
        final_res = final_res.query("weight > 0")
        final_res['request_id'] = request_id
        final_res['weight_percent'] = final_res['weight'] * 100
        final_res = final_res.sort_values('weight', ascending=False)
        final_res['index_no'] = [i + 1 for i in range(len(final_res))]
        final_res = final_res.drop('add', axis=1)
        # final_res['parametric_name'] = final_res['parametric_name'].str.replace("_", "+")
        final_res['PRODG1'] = final_res['PRODG1'].apply(lambda x: None if x == 'grplen2' else x)
        return final_res
    return df.groupby(by).apply(get_result)


In [5]:
##########################################################################################
#######################################对bad>=1的数据，用pca建模##############################
##########################################################################################
def fit_pca_small_sample(df, by):
    """
    param df: 小样本组合的数据
    param by: 分组字段
    return: PCA建模后的结果
    """
    schema_all = StructType([StructField("PRODG1", StringType(), True),
                             StructField("OPER_NO", StringType(), True),
                             StructField("EQP_NAME", StringType(), True),
                             StructField("features", StringType(), True),
                             StructField("importance", FloatType(), True),
                             StructField("bad_wafer", IntegerType(), True)])

    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_model_result(df_run):
        df_pivot = get_pivot_table(df=df_run, by=by)
        # 由于是小样本，再重新copy一份制造多一点数据传给PCA模型
        df_pivot_copy = df_pivot.copy()
        df_pivot_all = pd.concat([df_pivot, df_pivot_copy], axis=0)

        # 定义自变量
        x_train = df_pivot_all[df_pivot_all.columns.difference(['WAFER_ID', 'label']).tolist()]

        # 建立模型，传入给PCA的n_components选择x_train.shape中的最小值-1；
        # 选择是70%或者80%，出来的特征很有可能只是一两个
        model = pca(n_components=min(x_train.shape[0], x_train.shape[1]) - 1, verbose=None)
        results = model.fit_transform(x_train)
        res_top = results['topfeat']
        res_top_select = res_top[res_top['type'] == 'best'][['feature', 'loading']]
        res_top_select = res_top_select.drop_duplicates()
        res_top_select['importance'] = abs(res_top_select['loading'])
        res_top_select = res_top_select.rename(columns={'feature': 'features'})
        res_top_select = res_top_select.drop("loading", axis=1)

        # 增加一些字段信息
        res_top_select['bad_wafer'] = sum(df_pivot['label'])
        res_top_select['OPER_NO'] = df_run['OPER_NO'].values[0]
        res_top_select['EQP_NAME'] = df_run['EQP_NAME'].values[0]
        if len(by) == 3:
            res_top_select['PRODG1'] = df_run['PRODG1'].values[0]
        else:
            res_top_select['PRODG1'] = 'grplen2'

        return res_top_select

    return df.groupby(by).apply(get_model_result)


#####################################################################################
##################################对bad>=1建模后的结果进行整合############################
#####################################################################################
def split_calculate_features_small_sample(df, by):
    """
    param df: PCA建模后的结果
    param by: 分组字段
    return: features和importance结果
    """
    schema_all = StructType([StructField("PRODG1", StringType(), True),
                             StructField("OPER_NO", StringType(), True),
                             StructField("EQP_NAME", StringType(), True),
                             StructField("parametric_name", StringType(), True),
                             StructField("importance", FloatType(), True),
                             StructField("bad_wafer", FloatType(), True),
                             StructField("stats", StringType(), True)])

    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_result(model_results):
        feature_importance_table = model_results[['features', 'importance', 'bad_wafer']].dropna(axis=0)
        # 分裂features
        feature_importance_res_split = get_split_feature_importance_table(feature_importance_table, by)

        # 新增一列，含有参数的所有统计特征:feature_stats
        feature_stats = add_feature_stats(feature_importance_res_split)

        # 对同一种组合里的同一个参数进行求和:feature_importance_groupby
        feature_importance_groupby = (
            feature_importance_res_split.groupby(['PRODG1', 'OPER_NO', 'EQP_NAME', 'bad_wafer',
                                                  'parametric_name', 'step'])['importance'].sum().reset_index())
        feature_importance_groupby = feature_importance_groupby.assign(
            parametric_name=lambda x: x['parametric_name'] + str('#') + x['step']).drop('step', axis=1)

        # feature_stats和feature_importance_groupby连接
        grpby_stats = pd.merge(feature_stats, feature_importance_groupby,
                               on=['PRODG1', 'OPER_NO', 'EQP_NAME', 'parametric_name']).dropna().reset_index(drop=True)
        return grpby_stats

    return df.groupby(by).apply(get_result)


def get_finall_results_small_sample(f_res, bad_wafer_num):
    """
    param s_res: roc_auc分数结果
    param f_res: features和importance结果
    param bad_wafer_num: 数据中所有bad_wafer的数量
    return: 最后的建模结果
    """
    f_res = f_res.withColumn("bad_ratio", col("bad_wafer") / bad_wafer_num)
    df_merge = f_res.withColumn('weight_original', col('importance') * col('bad_ratio'))

    # 最后再次进行一次归一化
    weight_all = df_merge.agg({"weight_original": "sum"}).collect()[0][0]
    df_merge = df_merge.withColumn("weight", col("weight_original") / weight_all)

    df_merge = df_merge.select(['PRODG1', 'OPER_NO', 'EQP_NAME',
                                'parametric_name', 'weight', 'stats']).orderBy('weight', ascending=False)
    return df_merge


In [6]:
# 大样本数据模型整合
def fit_big_data_model(df_run, data_dict_list_bs, grpby_list, request_id):
    df1 = None
    df2 = None

    # 1. 获取用于建模的大样本数据
    df_run_bs = get_train_data(df_run, data_dict_list_bs)
    if df_run_bs.count() == 0:
        msg = '数据库中暂无此类数据!'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    # 2. 获取所有bad wafer数量
    bad_wafer_num_big_sample = get_all_bad_wafer_num(df_run_bs)
    if bad_wafer_num_big_sample < 3:
        msg = '数据库中实际BAD_WAFER数量小于3片, 请提供更多的BAD_WAFER数量!'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    # 3. 对挑选出的大样本数据进行建模
    res = fit_rf_big_sample(df=df_run_bs, by=grpby_list)
    if res.count() == 0:
        msg = '算法内部暂时异常!'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    # 4. 将建模结果进行整合
    s_res = split_score_big_sample(df=res, by=['PRODG1', 'OPER_NO', 'EQP_NAME'])
    if s_res.count() == 0:
        msg = '算法运行评分结果较低, 暂无输出, 建议增加BAD_WAFER数量'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    f_res = split_calculate_features_big_sample(df=res, by=grpby_list)
    if f_res.count() == 0:
        msg = '算法结果求和暂时异常'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    model_res_bs = get_finall_results_big_sample(s_res=s_res, f_res=f_res, bad_wafer_num=bad_wafer_num_big_sample)
    if model_res_bs.count() == 0:
        msg = '算法结果拼接暂时异常'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    # 7. 增加特定的列
    final_res_bs = model_res_bs.withColumn('add', lit(0))
    final_res_add_columns = add_certain_column(df=final_res_bs, by='add', request_id=request_id)
    if final_res_add_columns.count() == 0:
        msg = '算法结果增加列暂时异常'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2
    else:
        return df1, final_res_add_columns


In [7]:
# 小样本数据模型整合
def fit_small_data_model(df_run, common_res, grpby_list, request_id):
    df1 = None
    df2 = None

    data_dict_list_ss = get_data_list(common_res=common_res, grpby_list=grpby_list, big_or_small='small')
    print("data_dict_list_ss:", data_dict_list_ss)
    if len(data_dict_list_ss) == 0:
        msg = '该查询条件下数据库中实际BAD_WAFER数量为0, 无法分析'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    df_run_ss = get_train_data(df_run=df_run, data_dict_list=data_dict_list_ss)
    if df_run_ss.count() == 0:
        msg = '数据库中暂无此类数据!'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    bad_wafer_num_small_sample = get_all_bad_wafer_num(df_run_ss)
    if bad_wafer_num_small_sample < 1:
        msg = '该查询条件下数据库中实际BAD_WAFER数量小于1片, 请提供更多的BAD_WAFER数量!'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    res = fit_pca_small_sample(df=df_run_ss, by=grpby_list)
    if res.count() == 0:
        msg = '算法内部暂时异常!'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    f_res = split_calculate_features_small_sample(df=res, by=grpby_list)
    if f_res.count() == 0:
        msg = '算法结果求和暂时异常'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    model_res_ss = get_finall_results_small_sample(f_res=f_res, bad_wafer_num=bad_wafer_num_small_sample)
    if model_res_ss.count() == 0:
        msg = '算法结果拼接暂时异常'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2

    final_res_ss = model_res_ss.withColumn('add', lit(0))
    final_res_add_columns = add_certain_column(df=final_res_ss, by='add', request_id=request_id)
    if final_res_add_columns.count() == 0:
        msg = '算法结果增加列暂时异常'
        df_kafka = pd.DataFrame({"code": 1, "msg": f'{msg}', "requestId": request_id}, index=[0])
        df1 = spark.createDataFrame(df_kafka)
        return df1, df2
    else:
        return df1, final_res_add_columns

In [8]:
# grpby_list = ['OPER_NO', 'EQP_NAME']
grpby_list = ['PRODG1', 'OPER_NO', 'EQP_NAME']
request_id = 'sad'
merge_operno = []

In [15]:
df_pandas = pd.read_csv("D:/Jupyterfiles/晶合MVAFDC_general开发/MVAanlysisDevelop/uva_algorithm/CASE1_DATA/DWD_POC_CASE_FD_UVA_DATA_CASE1_PROCESSED1.csv")
# df_pandas = df_pandas.iloc[47000:51600, :]
print("读取数据pandas类型：", df_pandas.shape)


df1 = ps.from_pandas(df_pandas).to_spark()
print("pandas类型转为spark：", df1.count())

# 1. 站点融合和数据预处理
df1 = integrate_operno(df=df1, merge_operno_list=merge_operno)
print("站点融合后：", df1.count())

df_run = _pre_process(df1)
print("数据预处理后：", df_run.count())


# 2. 进行共性分析
common_res = commonality_analysis(df_run, grpby_list)
common_res.show()

读取数据pandas类型： (550911, 16)


D:\developer\Anaconda\lib\site-packages\pyspark\pandas\internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
D:\developer\Anaconda\lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


pandas类型转为spark： 550911
站点融合后： 550911
数据预处理后： 550896
+--------+--------+--------+-----------+--------+-------+
|  PRODG1| OPER_NO|EQP_NAME|wafer_count|good_num|bad_num|
+--------+--------+--------+-----------+--------+-------+
|L11TG07A|1F.EEK10|   EKT72|       1000|       0|   1000|
|L11CD02A|1F.EEK10|   EKT72|        959|     234|    725|
|L11TG05A|1F.EEK10|   EKT72|        474|     175|    299|
|L11EG07A|1F.EEK10|   EKT72|        224|       0|    224|
|L15KD03A|1F.EEK10|   EKT72|        101|       0|    101|
|L11DM10A|1F.EEK10|   EKT72|         75|       0|     75|
|L15RB03A|1F.EEK10|   EKT72|        175|     100|     75|
|L15DV07A|1F.EEK10|   EKT72|         76|       1|     75|
|L11MW20A|1F.EEK10|   EKT72|         75|       1|     74|
|L11CB14A|1F.EEK10|   EKT72|        225|     200|     25|
+--------+--------+--------+-----------+--------+-------+



In [16]:
# 3. 挑选出数据：bad和good要同时大于3
data_dict_list_bs = get_data_list(common_res, grpby_list, big_or_small='big')
print("data_dict_list_bs:", data_dict_list_bs)

if len(data_dict_list_bs) != 0:
    print("****************大样本算法调用****************")
    df1, final_res_add_columns = fit_big_data_model(df_run, data_dict_list_bs, grpby_list, request_id)
else:
    print("****************小样本算法调用****************")
    df1, final_res_add_columns = fit_small_data_model(df_run, common_res, grpby_list, request_id)

data_dict_list_bs: [{'PRODG1': 'L11CD02A', 'OPER_NO': '1F.EEK10', 'EQP_NAME': 'EKT72'}, {'PRODG1': 'L11TG05A', 'OPER_NO': '1F.EEK10', 'EQP_NAME': 'EKT72'}, {'PRODG1': 'L15RB03A', 'OPER_NO': '1F.EEK10', 'EQP_NAME': 'EKT72'}, {'PRODG1': 'L11CB14A', 'OPER_NO': '1F.EEK10', 'EQP_NAME': 'EKT72'}]
****************大样本算法调用****************


D:\developer\Anaconda\lib\site-packages\pyspark\sql\pandas\group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [17]:
df1.toPandas()

AttributeError: 'NoneType' object has no attribute 'toPandas'

In [18]:
print("全部数据建模结果：")
final_res_add_columns.show()

全部数据建模结果：
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|  PRODG1| OPER_NO|EQP_NAME|     stats|     parametric_name|     weight|request_id|weight_percent|index_no|
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|L11CD02A|1F.EEK10|   EKT72|MEAN#SLOPE|LO_RF_VPP#AOTU_ST...| 0.10327881|       sad|     10.327881|       1|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_VPP#STEP2_MINI|0.083826795|       sad|       8.38268|       2|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_VPP#STEP2_M...| 0.06990923|       sad|     6.9909234|       3|
|L11TG05A|1F.EEK10|   EKT72|  MAX#MEAN|ESC_CURRENT#AOTU_...| 0.05286586|       sad|      5.286586|       4|
|L11CD02A|1F.EEK10|   EKT72|  MEAN#MAX|LO_RF_REF_POWER#A...|0.042609613|       sad|     4.2609615|       5|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|PROCESS_GAS_8_O2#...| 0.04184937|       sad|      4.184937|       6|
|L11CD02A|1F.EEK10

In [25]:
print("1:500建模结果：")
final_res_add_columns.show()

1:500建模结果：
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|  PRODG1| OPER_NO|EQP_NAME|     stats|     parametric_name|     weight|request_id|weight_percent|index_no|
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|L11CD02A|1F.EEK10|   EKT72|      MEAN|PROCESS_GAS_8_O2#...| 0.16348487|       sad|     16.348486|       1|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_VPP#STEP2_M...| 0.16033822|       sad|     16.033821|       2|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_VPP#STEP2_MINI| 0.10415568|       sad|     10.415568|       3|
|L11CD02A|1F.EEK10|   EKT72|MEAN#SLOPE|LO_RF_VPP#AOTU_ST...| 0.10382607|       sad|    10.3826065|       4|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|EDGE_GAS_PRESSURE...| 0.09637696|       sad|      9.637696|       5|
|L11CD02A|1F.EEK10|   EKT72|       SUM|PROCESS_GAS_5_CHF...| 0.07881552|       sad|      7.881552|       6|
|L11CD02A|1F.EEK1

In [29]:
print("1000:2000建模结果：")
final_res_add_columns.show()

1000:2000建模结果：
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|  PRODG1| OPER_NO|EQP_NAME|     stats|     parametric_name|     weight|request_id|weight_percent|index_no|
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|L11CD02A|1F.EEK10|   EKT72|MEAN#SLOPE|LO_RF_VPP#AOTU_ST...| 0.13653621|       sad|     13.653622|       1|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LOWER_TEMPERATURE...|  0.1221142|       sad|     12.211419|       2|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|EDGE_HE_FLOW#AOTU...| 0.11701586|       sad|     11.701586|       3|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_VPP#STEP2_MINI| 0.10394778|       sad|     10.394778|       4|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|WALL_TEMPERATURE#...| 0.07280642|       sad|      7.280642|       5|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_VPP#STEP2_M...| 0.07248003|       sad|      7.248003|       6|
|L11CD02A|1F.

In [14]:
print("47000:51600建模结果：")
final_res_add_columns.show()

47000:51600建模结果：
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|  PRODG1| OPER_NO|EQP_NAME|     stats|     parametric_name|     weight|request_id|weight_percent|index_no|
+--------+--------+--------+----------+--------------------+-----------+----------+--------------+--------+
|L11CD02A|1F.EEK10|   EKT72|      MEAN|EDGE_GAS_PRESSURE...| 0.08960889|       sad|       8.96089|       1|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_VPP#STEP2_MINI|  0.0773143|       sad|       7.73143|       2|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LOWER_TEMPERATURE...| 0.07480955|       sad|     7.4809546|       3|
|L11CD02A|1F.EEK10|   EKT72|MEAN#SLOPE|LO_RF_VPP#AOTU_ST...|0.070233785|       sad|      7.023379|       4|
|L11CD02A|1F.EEK10|   EKT72|  MEAN#MAX|LO_RF_REF_POWER#A...|  0.0679332|       sad|     6.7933207|       5|
|L11CD02A|1F.EEK10|   EKT72|      MEAN|LO_RF_POWER#AOTU_...|0.065307885|       sad|      6.530789|       6|
|L11CD02A|1